In [1]:
import sqlite3
import pandas as pd

In [2]:
pd.options.display.max_columns = None
#pd.options.display.max_rows = None

In [3]:
conn = sqlite3.connect('parch-and-posey.db')

In [4]:
cursor = conn.cursor()
cursor.execute('''
select * from sqlite_master where type = "table";
''')
columns = [col[0] for col in cursor.description]
data = cursor.fetchall()
cursor.close()

In [5]:
pd.DataFrame(data, columns=columns)

,type,name,tbl_name,rootpage,sql
0,table,web_events,web_events,2,"CREATE TABLE web_events (\tid integer,\taccoun..."
1,table,sales_reps,sales_reps,92,"CREATE TABLE sales_reps (\tid integer,\tname b..."
2,table,region,region,93,"CREATE TABLE region (\tid integer,\tname bpchar)"
3,table,orders,orders,94,"CREATE TABLE orders (\tid integer,\taccount_id..."
4,table,accounts,accounts,221,"CREATE TABLE accounts (\tid integer,\tname bpc..."


# EXPLAIN QUERY PLAN

In [7]:
# sqlite version

pd.read_sql_query(sql = '''
EXPLAIN QUERY PLAN
SELECT strftime('%Y-%m-%d', o.occurred_at) AS date,
COUNT(DISTINCT a.sales_rep_id) AS active_sales_reps,
COUNT(DISTINCT o.id) AS orders,
COUNT(DISTINCT we.id) AS web_visits
FROM accounts a
JOIN orders o
ON o.account_id = a.id
JOIN web_events we
ON strftime('%Y-%m-%d', we.occurred_at) = strftime('%Y-%m-%d', o.occurred_at)
GROUP BY 1
ORDER BY 1 DESC
;
''', con=conn)

,id,parent,notused,detail
0,9,0,0,SCAN TABLE accounts AS a
1,11,0,0,SCAN TABLE web_events AS we
2,23,0,0,SEARCH TABLE orders AS o USING AUTOMATIC COVER...
3,34,0,0,USE TEMP B-TREE FOR GROUP BY


In [8]:
# sqlite version - table is big without aggregate - 79083 rows × 4 columns

pd.read_sql_query(sql = '''
EXPLAIN QUERY PLAN
SELECT o.occurred_at AS date,
a.sales_rep_id AS active_sales_reps,
o.id AS orders,
we.id AS web_visits
FROM accounts a
JOIN orders o
ON o.account_id = a.id
JOIN web_events we
ON strftime('%Y-%m-%d', we.occurred_at) = strftime('%Y-%m-%d', o.occurred_at)
ORDER BY 1 DESC
;
''', con=conn)

,id,parent,notused,detail
0,5,0,0,SCAN TABLE accounts AS a
1,7,0,0,SCAN TABLE web_events AS we
2,19,0,0,SEARCH TABLE orders AS o USING AUTOMATIC COVER...
3,39,0,0,USE TEMP B-TREE FOR ORDER BY


In [9]:
# sqlite version - aggregate the tables separately

pd.read_sql_query(sql = '''
EXPLAIN QUERY PLAN
SELECT COALESCE(orders.date, web_events.date),
COUNT(DISTINCT orders.active_sales_reps) AS active_sales_reps,
COUNT(DISTINCT orders.orders) AS orders,
COUNT(DISTINCT web_events.web_visits) AS web_visits
FROM (SELECT strftime('%Y-%m-%d', o.occurred_at) AS date,
COUNT(a.sales_rep_id) AS active_sales_reps,
COUNT(o.id) AS orders
FROM accounts a
JOIN orders o
ON o.account_id = a.id
GROUP BY 1) orders
JOIN
(SELECT strftime('%Y-%m-%d', we.occurred_at) AS date,
COUNT(we.id) AS web_visits
FROM web_events we
GROUP BY 1) web_events
ON orders.date = web_events.date
GROUP BY 1
ORDER BY 1 DESC

;
''', con=conn)

,id,parent,notused,detail
0,3,0,0,MATERIALIZE 1
1,11,3,0,SCAN TABLE accounts AS a
2,23,3,0,SEARCH TABLE orders AS o USING AUTOMATIC COVER...
3,29,3,0,USE TEMP B-TREE FOR GROUP BY
4,79,0,0,MATERIALIZE 2
5,86,79,0,SCAN TABLE web_events AS we
6,88,79,0,USE TEMP B-TREE FOR GROUP BY
7,135,0,0,SCAN SUBQUERY 1 AS orders
8,146,0,0,SEARCH SUBQUERY 2 AS web_events USING AUTOMATI...
9,151,0,0,USE TEMP B-TREE FOR GROUP BY


# Additional Practice Resources

If you would like to get more practice writing SQL queries, there are several great websites to practice writing SQL queries. Here are a couple we recommend: HackerRank https://www.hackerrank.com/domains/sql and ModeAnalytics https://mode.com/sql-tutorial/sql-business-analytics-training/. We strongly recommend these. The skill test by AnalyticsVidhya is a fun test to take too https://www.analyticsvidhya.com/blog/2017/01/46-questions-on-sql-to-test-a-data-science-professional-skilltest-solution/.

You will need to create a profile for HackerRank and create an account for Mode Analytics, but these are excellent routes to gain more practice and learn more advanced skills along the way. If you come across exercises that require knowledge on concepts you haven't learned yet, feel free to google them. Spending time practicing, making mistakes and learning from it - that is the best way to become a master at anything!

See you in the next lesson!